### 비공개 애트리뷰트보다는 공개 애트리뷰트를 사용하라.
- 파이썬에서 클래스의 애트리뷰트에 대한 가시성은 공개와 비공개 두가지밖에 없다.

In [1]:
class MyObject:
    def __init__(self):
        self.public_field = 5
        self.__private_field = 10

    def get_private_field(self):
        return self.__private_field


- 객체 뒤에 점 연산자(.)를 붙이면 공개 애트리뷰트에 접근할 수 있다.

In [4]:
foo = MyObject()
print(foo.public_field)
assert foo.public_field == 5

5


- 애트리뷰트 이름 앞에 밑줄 두 개(__) 붙이면 비공개 필드가 된다.
- 비공개 필그를 포함하는 클래스 안에 있는 메서드에서는 해당 필드에 직접 접근할 수 있다.

In [3]:
assert foo.get_private_field() == 10

- 하지만 클래스 외부에서 비공개 필드에 접근하면 예외 발생

In [5]:
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
foo.__private_field

AttributeError: 'MyObject' object has no attribute '__private_field'

- 클래스 매서드는 자신이 둘러싸고 있는 class 블록 내부에 들어 있기 때문에 해당 클래스의 비공개 필드에 접근할 수 있다.

In [6]:
class MyOtherObject:
    def __init__(self):
        self.__private_field = 71

    @classmethod
    def get_private_field_of_instance(cls, instance):
        return instance.__private_field

bar = MyOtherObject()
assert MyOtherObject.get_private_field_of_instance(bar) == 71

- 예상했겠지만 하위 클래스는 부모 클래스의 비공개 필드에 접근할 수 있다.

In [9]:
class MyParentObject:
    def __init__(self):
        self.__private_field = 71

class MyChildObject(MyParentObject):
    def get_private_field(self):
        return self.__private_field

baz = MyChildObject()
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
#baz.get_private_field()

- 비공개 애트리뷰트의 동작은 애트리뷰트 이름을 바꾸는 단순한 방식으로 구현된다.
- MyChildObject.get_private_field처럼 메서드 내부에서 비공개 애트리뷰트에 접근하는 코드가 있으면, 파이썬 컴파일러는 $__private_field$라는 애트리뷰트 접근 코드를 _MyChildObject__private_field라는 이름으로 바꿔 준다.
- 위 예제에서는 $MyParentObject.__init__$안에만 $__private_fieild$ 정의가 들어 있다.
- 이는 이 비공개 필드의 이름이 실제로는 $_MyParentObject__private_field$라는 뜻이다.
- 부모의 비공개 애트리뷰트를 자식 애트리뷰트에서 접근하면, 단지 변경한 애트리뷰트 이름이 존재 하지 않는다는 이유로 오류가 발생한다.

- 이 방식을 알고 나면 특별한 권한을 요청할 필요 없이 쉽게, 하위 클래스에서든 클래스 외부에서든 원하는 클래스의 비공개 애트리뷰트에 접근할 수 있다.

In [10]:
assert baz._MyParentObject__private_field == 71

- 객체 애트리뷰트 딕셔너리를 살펴보면 실제로 변환된 비공개 애트리뷰트 이름이 들어 있는 모습을 볼 수 있다.

In [11]:
print(baz.__dict__)

{'_MyParentObject__private_field': 71}


비공개 애트리뷰트에 대한 접근 구문이 실제로 가시성을 엄격하게 제한하지 않는 이유
- 파이썬의 모토로 자주 회자되는 '우리는 모두 책임질줄 아는 성인이다'일 것이다.
- 특정 기능을 확장할지 여부는 그러한 위험을 택함으로써 야기될 수 있는 결과에 대해 책임지는 한 우리의 선택일 뿐이다.
- 파이썬 프로그래머들은 열어둠으로써 얻을 수 있는 이익이 그로인해 발생할 수 있는 해악보다 더 크다고 믿는다.

내부를 몰래 접근함으로써 생길 수 있는 피해를 줄이고자 파이썬 프로그래머는 스타일 가이드에 정해진 명명 규약을 지킨다.
- 필드 앞에 밑줄이 하나만 있으면(_protected_field) 관례적 보호필드를 뜻한다.
- 보호 필드는 클래스 외부에서 이 필드를 사용하는 경우 조심해야 한다는 뜻이다.
- 하지만 파이썬을 처음 사용하는 많은 프로그래머가 하위 클래스나 클래스 외부에서 사용하면 안 되는 내부 API를 표현하기 위해 비공개 필드를 사용한다.

In [12]:
class MyStringClass:
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return str(self.__value)

foo = MyStringClass(5)
assert foo.get_value() == '5'

위 접근 방법은 잘못된 것이다.
- 누군가는 이 클래스를 상송하면서 새로운 기능을 추가하거나, 기존 메서드의 단점을 해결하기 위해 새로운 동작을 추가하길 원할 수 있다.
- 예를 들어 MyStringClass.get_value가 항상 문자열을 반환하는데, 이를 바꾸고 싶을 수도 있다.
- 비공개 애트리뷰트를 사용하면 이런 확장이나 하위 클래스의 오버라이드를 귀찮게 하고 깨지기 쉽게 만들 뿐이다.
- 여러분이 만든 클래스의 하위 클래스를 만드는 사람이 비공개 필드에 꼭 접근해야 한다면 여전히 비공개 필드에 접근할 수 있다.


In [ ]:
class MyIntegerSubclass(MyStringClass):
    def get_value(self):
        return int(self._MyStringClass__value)

foo = MyIntegerSubclass('5')
assert foo.get_value() == 5

- 자신의 클래스 정의를 변경하면 더 이상 비공개 애트리뷰트에 대한 참조가 바르지 않으므로 하위 클래스가 깨질 것이다.
- 다음은 MyIntegerSubclass의 부모 클래스인 MyStringClass에 새로운 부모 클래스인 MyBaseClass를 추가한 코드다.

In [15]:
class MyBaseClass:
    def __init__(self, value):
        self.__value = value

    def get_value(self):
        return self.__value

class MyStringClass(MyBaseClass):
    def get_value(self):
        return str(super().get_value())  # 변경됨

class MyIntegerSubclass(MyStringClass):
    def get_value(self):
        return int(self._MyStringClass__value)  # 변경되지 않음

- 이제 MyStringClass가 아닌 MyBaseClass 클래스에서 __value 애트리뷰트를 할당한다.
- 이로 인해 MyIntegerSubclass 안에 있는 비공개 변수에 대한 참조인 self._MyStringClass__value가 깨진다.

In [16]:
foo = MyIntegerSubclass(5)
# 오류가 나는 부분. 오류를 보고 싶으면 커멘트를 해제할것
foo.get_value()

AttributeError: 'MyIntegerSubclass' object has no attribute '_MyStringClass__value'

- 비공개 애트리뷰트를 사용할지 진지하게 고민해야 하는 유일한 경우는 하위 클래스의 필드와 이름이 충돌할 수 있는 경우뿐이다.
- 자식 클래스가 실수로 부모 클래스가 이미 정의한 애트리뷰트를 정의하면 충돌이 생길 수 있다.


In [17]:
class ApiClass:
    def __init__(self):
        self._value = 5

    def get(self):
        return self._value

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello'  # 충돌

a = Child()
print(f'{a.get()} 와 {a._value} 는 달라야 합니다.')

hello 와 hello 는 달라야 합니다.


부모 클래스 쪽에서 자식 클래스의 애트리뷰트 이름이 자신의 애트리뷰트 이름과 겹치는 일을 방지하기 위해 비공개 애트리뷰트를 사용할 수 있다.

In [18]:
class ApiClass:
    def __init__(self):
        self.__value = 5    # 밑줄 2개!

    def get(self):
        return self.__value # 밑줄 2개!

class Child(ApiClass):
    def __init__(self):
        super().__init__()
        self._value = 'hello' # OK!

a = Child()
print(f'{a.get()} 와 {a._value} 는 다릅니다.')

5 와 hello 는 다릅니다.
